---

# 1. Introduction
In this notebook, we will analyse the following betting strategies:
1. **Betting on the Favourite**(Team With Lower Odds)
2. **Betting on the Underdog**(Team with Higher Odds)
3. **Betting on a Draw**

For our purposes, we will merge the two tables from our SQLite database based off each fixture's unique `fixture_id` to form a table with the odds and match outcomes. We will then pick the best possible odds for our respective strategies for analysis.

---

# 2. Import Libraries
We import the necessary libraries for:
- Data manipulation using `pandas`.
- Interacting with the SQL database using `SQLAlchemy`.
- Generating and saving our visualisations with `lets_plot`.

In [1]:
import pandas as pd
from sqlalchemy import create_engine
from lets_plot import * 
from lets_plot import ggsave
from IPython.display import SVG

---

# 3. Merging Odds Data and Match Outcomes 

In [2]:
engine = create_engine('sqlite:///../data/sports_odds.db')
odds_df = pd.read_sql('SELECT * FROM historical_odds', con=engine)
results_df = pd.read_sql('SELECT * FROM match_results', con=engine)

After inspection, we realise that there is a different number of rows for `odds_df` and `results_df`. These discrepancies in the data size are normal and will not affect our analysis. Hence, we used `pandas` merge on 'inner' to ensure only matches with `fixture_id` present in both dataframes will be selected for analysis.

In [15]:
merged_df = odds_df.merge(results_df, on='fixture_id', how='inner')
merged_df.head()

,match_id,commence_time,home_team,away_team,Unibet_home_odds,Unibet_away_odds,Unibet_draw_odds,Sky Bet_home_odds,Sky Bet_away_odds,Sky Bet_draw_odds,...,Grosvenor_draw_odds,Smarkets_home_odds,Smarkets_away_odds,Smarkets_draw_odds,fixture_id,Date,Time,HomeTeam,AwayTeam,FTR
0,2dd4a4f8663e6f835226a5209c614a60,2020-06-17T17:00:00Z,Aston Villa,Sheffield United,3.35,2.32,3.25,3.10,2.25,3.30,...,NaN,NaN,NaN,NaN,AVLSHU170620,17/06/2020,18:00,Aston Villa,Sheffield United,D
1,b1e029a0d989b4c11e843204003044f9,2020-06-17T19:15:00Z,Manchester City,Arsenal,1.35,8.50,5.60,1.36,7.50,5.25,...,NaN,NaN,NaN,NaN,MCIARS170620,17/06/2020,20:15,Man City,Arsenal,H
2,59d68295dc2213634772cd941c91fa11,2020-06-19T19:15:00Z,Tottenham Hotspur,Manchester United,2.75,2.60,3.30,2.70,2.50,3.40,...,NaN,NaN,NaN,NaN,TOTMUN190620,19/06/2020,20:15,Tottenham,Man United,D
3,88352746f45f6beb4e2cb662d9414d0f,2020-06-20T11:30:00Z,Watford,Leicester City,3.40,2.15,3.45,3.25,2.20,3.40,...,NaN,NaN,NaN,NaN,WATLEI200620,20/06/2020,12:30,Watford,Leicester,D
4,065ae59da20562892de52b7f5598ecbf,2020-06-20T16:30:00Z,West Ham United,Wolverhampton Wanderers,3.50,2.15,3.35,3.30,2.20,3.30,...,NaN,NaN,NaN,NaN,WHUWOL200620,20/06/2020,17:30,West Ham,Wolves,A


---